In [1]:
from graphtoolbox import OgbDataHelper
from ogb.graphproppred import GraphPropPredDataset
import numpy as np

/home/cstaib/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ogb_data_helper = OgbDataHelper()

In [3]:
dataset_name = 'ogbg-molpcba'

In [4]:
dataset_dict = ogb_data_helper.get_processed_dataset(dataset_name)

100%|██████████| 437929/437929 [02:03<00:00, 3535.62it/s]


In [5]:
dataset = GraphPropPredDataset(name = dataset_name)

split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]

: 

: 

In [ ]:
dataset[np.array(train_idx)]